In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

2024-01-06 10:22:11.836215: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-06 10:22:12.036428: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-06 10:22:12.755184: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-01-06 10:22:12.755269: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

# creating classifier

In [2]:
autoencoder_model = tf.keras.models.load_model(
    r"./saved_models/encoder_v3",
    custom_objects={"custom_loss": None},
)

autoencoder_model.trainable = False
model_input = autoencoder_model.layers[0].input
encoder_output = autoencoder_model.get_layer("encoder_output").output
encoder = keras.Model(inputs=[model_input], outputs=[encoder_output])

x = x_in = keras.layers.Input((256, 256, 3))

encoder_output = encoder(x)


x = keras.layers.Flatten()(encoder_output)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Dense(5, "softmax")(x)

# create model
classifier = keras.Model(inputs=[x_in], outputs=[x])

classifier.summary()

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


2024-01-06 10:22:13.888461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-06 10:22:13.921639: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2024-01-06 10:22:13.921660: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-01-06 10:22:13.922857: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 model (Functional)          (None, 16, 16, 16)        6840      
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 5)                 20485     
                                                                 
Total params: 27,325
Trainable params: 20,485
Non-trainable params: 6,840
_________________________________________________________________


# load classifier

In [3]:
classifier = tf.keras.models.load_model(r"./saved_models/classifier_v2")

classifier.summary(expand_nested=True)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 model_2 (Functional)        (None, 16, 16, 16)        6840      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_layer (InputLayer)  [(None, 256, 256, 3)]     0         |
|                                                               |
| depthwise_conv2d (Depthwise  (None, 16, 16, 24)     6168      |
| Conv2D)                                                       |
|                                                               |
| conv2d (Conv2D)           (None, 16, 16, 16)        400       |
|                                                               |
| encoder_output (Conv2D)   (None, 16, 16, 16)        272       |
¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯

In [4]:
for l in classifier.layers:
    if isinstance(l, tf.keras.Model):
        for l2 in l.layers:
            print(l2)

    print(l.name)

input_1
model_2
flatten
dropout
dense


# Save model weights as numpy file

the name of the file indicates the 'type' of the layer, the layer 'name', and 'activation', separeted by '.'.\
eg: Conv2D.encoder_output.Linear

In [32]:
def save_layer_file(id, layer):
    layer_name = layer.name
    layer_type = None
    activation_type = "Linear"
    weights, bias = [[], []]

    if isinstance(layer, tf.keras.layers.Conv2D):
        layer_type = "Conv2D"
        weights, bias = layer.get_weights()
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer_type = "DepthwiseConv2D"
        weights, bias = layer.get_weights()
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.Dense):
        layer_type = "Dense"
        weights, bias = layer.get_weights()
        activation_type = layer.get_config()["activation"]

    elif isinstance(layer, tf.keras.layers.Flatten):
        layer_type = "Flatten"

    elif isinstance(layer, tf.keras.layers.InputLayer):
        layer_type = "InputLayer"

    else:
        print(f"layer type {type(layer)} not expected..")


    np.save(
        f"./saved_models/classifier_v2/{id}.{layer_type}.{layer_name}.{activation_type}.weights",
        weights,
    )
    np.save(
        f"./saved_models/classifier_v2/{id}.{layer_type}.{layer_name}.{activation_type}.bias",
        bias,
    )


index = 0
for l in classifier.layers:
    if isinstance(l, tf.keras.Model):

        for l2 in l.layers[1:]:
            save_layer_file(index, l2)

            index += 1
    else:
        print(l.name)
        save_layer_file(index, l)
        index += 1

input_1
flatten
dropout
layer type <class 'keras.layers.regularization.dropout.Dropout'> not expected..
dense


In [6]:
denseLayer = classifier.get_layer(name="dense")

weights, bias = denseLayer.get_weights()

np.save("./saved_models/classifier_v2/dense_weights", weights)
np.save("./saved_models/classifier_v2/dense_bias", bias)

# create the numpy layer

In [7]:
from Layers import Dense, Softmax

softmax = Softmax()
fc = Dense("dense", activation=softmax)
fc.load_weights(
    "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/6.Dense.dense.softmax.weights.npy",
    "/home/george/Desktop/dnn_processor/reference_algorithm/neural_network/saved_models/classifier_v2/6.Dense.dense.softmax.bias.npy",
)

ModuleNotFoundError: No module named 'reference_algorithm'

# test 

In [ ]:
# test the tensorflow layer and the numpy layer on the same input

input_test = np.zeros((32,4096))

print(fc(input_test, False))
print(denseLayer(input_test).numpy())